📌 week5 내용 주차에 해당되는 과제는 Glove 모델 실습, NER task 실습, Dependency Parsing task 실습으로 구성되어 있습니다. (**참고** : **제출은 week6 branch 복습과제로!**)

📌 위키독스의 딥러닝을 이용한 자연어 처리 입문 교재 실습, 캐글 노트북 등의 자료로 구성되어있는 과제입니다. 

📌 안내된 링크에 맞추어 **직접 코드를 따라 치면서 (필사)** 해당 nlp task 의 기본적인 라이브러리와 메서드를 숙지해보시면 좋을 것 같습니다😊 필수라고 체크한 부분은 과제에 반드시 포함시켜주시고, 선택으로 체크한 부분은 자율적으로 스터디 하시면 됩니다.

📌 궁금한 사항은 깃허브 이슈나, 카톡방, 세션 발표 시작 이전 시간 등을 활용하여 자유롭게 공유해주세요!

In [1]:
import nltk
# nltk colab 환경에서 실행시 필요한 코드입니다. 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

### 1️⃣ **Glove**



👀 **내용 복습** 
* 스탠포드 대학에서 개발한 카운트 기반과 예측 기반을 모두 사용하는 단어 임베딩 방법론 
* word2vec 의 단점을 보완해서 나온 모델 
* glove model 의 **input 은 반드시 동시등장행렬 형태**여야 한다 ⭐

![1](https://www.dropbox.com/s/nz0ji4yzre56ifv/word_presentation.png?raw=1) 




🤔 한국어 예제는 없는 것 같습니다. 논문에서는 k-Glove 로 소개되는 연구가 있긴 한데, 좀 더 알아봐야 할 것 같아요!

➕ [논문1](https://scienceon.kisti.re.kr/srch/selectPORSrchArticle.do?cn=NPAP13255003&dbt=NPAP)


➕[논문2](https://scienceon.kisti.re.kr/commons/util/originalView.do?cn=CFKO201832073078664&oCn=NPAP13255064&dbt=CFKO&journal=NPRO00383361&keyword=%ED%95%9C%EA%B5%AD%EC%96%B4%20%EB%8C%80%ED%99%94%20%EC%97%94%EC%A7%84%EC%97%90%EC%84%9C%EC%9D%98%20%EB%AC%B8%EC%9E%A5%EB%B6%84%EB%A5%98)

🔹 **1-(1)** glove python

* [실습 : basic code](https://wikidocs.net/22885) 👉 필수

In [2]:
pip install glove_python_binary


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 948 kB 5.1 MB/s 


In [10]:
from glove import Corpus, Glove

corpus = Corpus() 

corpus.fit(result, window=5)
glove = Glove(no_components=100, learning_rate=0.05)


glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

NameError: ignored

In [7]:
print(glove.most_similar("man"))

NameError: ignored

🔹 **1-(2)** pre-trained glove 

* **사전학습모델** : 임의의 값으로 초기화하던 모델의 가중치들을 다른 문제에 학습시킨 가중치들로 초기화하는 방법이다.사전 학습한 가중치를 활용해 학습하고자 하는 본래 문제를 하위문제라고 한다. 

* [실습 : 문장의 긍부정을 판단하는 감성 분류 모델 만들기](https://wikidocs.net/33793) 👉 필수
  * [설명참고](https://omicro03.medium.com/%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC-nlp-16%EC%9D%BC%EC%B0%A8-pre-trained-word-embedding-bb30db424a35)
* pre-trained data 를 가져오는데 시간이 오래걸림
* kaggle 대회에서 주로 이 방식을 많이 사용함
  * [참고](https://lsjsj92.tistory.com/455)

In [11]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

sentences = ['nice great best amazing', 'stop lies', 'pitiful nerd', 'excellent work', 'supreme quality', 'bad', 'highly respectable']
y_train = [1, 0, 0, 1, 1, 0, 1]

In [13]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(sentences)
vocab_size = len(tokenizer.word_index) + 1 # 패딩을 고려하여 +1
print('단어 집합 :',vocab_size)

단어 집합 : 16


In [14]:
X_encoded = tokenizer.texts_to_sequences(sentences)
print('정수 인코딩 결과 :',X_encoded)

정수 인코딩 결과 : [[1, 2, 3, 4], [5, 6], [7, 8], [9, 10], [11, 12], [13], [14, 15]]


In [15]:
max_len = max(len(l) for l in X_encoded)
print('최대 길이 :',max_len)

최대 길이 : 4


In [16]:
X_train = pad_sequences(X_encoded, maxlen=max_len, padding='post')
y_train = np.array(y_train)
print('패딩 결과 :')
print(X_train)

패딩 결과 :
[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

embedding_dim = 4

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

Epoch 1/100
1/1 - 1s - loss: 0.7020 - acc: 0.4286 - 849ms/epoch - 849ms/step
Epoch 2/100
1/1 - 0s - loss: 0.7005 - acc: 0.4286 - 8ms/epoch - 8ms/step
Epoch 3/100
1/1 - 0s - loss: 0.6990 - acc: 0.4286 - 7ms/epoch - 7ms/step
Epoch 4/100
1/1 - 0s - loss: 0.6976 - acc: 0.4286 - 8ms/epoch - 8ms/step
Epoch 5/100
1/1 - 0s - loss: 0.6961 - acc: 0.5714 - 9ms/epoch - 9ms/step
Epoch 6/100
1/1 - 0s - loss: 0.6947 - acc: 0.5714 - 9ms/epoch - 9ms/step
Epoch 7/100
1/1 - 0s - loss: 0.6933 - acc: 0.5714 - 7ms/epoch - 7ms/step
Epoch 8/100
1/1 - 0s - loss: 0.6918 - acc: 0.5714 - 7ms/epoch - 7ms/step
Epoch 9/100
1/1 - 0s - loss: 0.6904 - acc: 0.5714 - 5ms/epoch - 5ms/step
Epoch 10/100
1/1 - 0s - loss: 0.6890 - acc: 0.5714 - 8ms/epoch - 8ms/step
Epoch 11/100
1/1 - 0s - loss: 0.6876 - acc: 0.5714 - 7ms/epoch - 7ms/step
Epoch 12/100
1/1 - 0s - loss: 0.6862 - acc: 0.5714 - 7ms/epoch - 7ms/step
Epoch 13/100
1/1 - 0s - loss: 0.6847 - acc: 0.5714 - 7ms/epoch - 7ms/step
Epoch 14/100
1/1 - 0s - loss: 0.6833 - acc:

In [18]:
print(X_train)

[[ 1  2  3  4]
 [ 5  6  0  0]
 [ 7  8  0  0]
 [ 9 10  0  0]
 [11 12  0  0]
 [13  0  0  0]
 [14 15  0  0]]


In [19]:
print(y_train)

[1 0 0 1 1 0 1]


In [20]:
from urllib.request import urlretrieve, urlopen
import gzip
import zipfile

urlretrieve("http://nlp.stanford.edu/data/glove.6B.zip", filename="glove.6B.zip")
zf = zipfile.ZipFile('glove.6B.zip')
zf.extractall() 
zf.close()

In [21]:
embedding_dict = dict()

f = open('glove.6B.100d.txt', encoding="utf8")

for line in f:
    word_vector = line.split()
    word = word_vector[0]

    word_vector_arr = np.asarray(word_vector[1:], dtype='float32')
    embedding_dict[word] = word_vector_arr
f.close()

print('%s개의 Embedding vector가 있습니다.' % len(embedding_dict))

400000개의 Embedding vector가 있습니다.


In [22]:
print(embedding_dict['respectable'])
print('벡터의 차원 수 :',len(embedding_dict['respectable']))

embedding_matrix = np.zeros((vocab_size, 100))
print('임베딩 행렬의 크기(shape) :',np.shape(embedding_matrix)

print(tokenizer.word_index.items())

print('단어 great의 맵핑된 정수 :',tokenizer.word_index['great']
      
print(embedding_dict['great'])



[-0.049773   0.19903    0.10585    0.1391    -0.32395    0.44053
  0.3947    -0.22805   -0.25793    0.49768    0.15384   -0.08831
  0.0782    -0.8299    -0.037788   0.16772   -0.45197   -0.17085
  0.74756    0.98256    0.81872    0.28507    0.16178   -0.48626
 -0.006265  -0.92469   -0.30625   -0.067318  -0.046762  -0.76291
 -0.0025264 -0.018795   0.12882   -0.52457    0.3586     0.43119
 -0.89477   -0.057421  -0.53724    0.25587    0.55195    0.44698
 -0.24252    0.29946    0.25776   -0.8717     0.68426   -0.05688
 -0.1848    -0.59352   -0.11227   -0.57692   -0.013593   0.18488
 -0.32507   -0.90171    0.17672    0.075601   0.54896   -0.21488
 -0.54018   -0.45882   -0.79536    0.26331    0.18879   -0.16363
  0.3975     0.1099     0.1164    -0.083499   0.50159    0.35802
  0.25677    0.088546   0.42108    0.28674   -0.71285   -0.82915
  0.15297   -0.82712    0.022112   1.067     -0.31776    0.1211
 -0.069755  -0.61327    0.27308   -0.42638   -0.085084  -0.17694
 -0.0090944  0.1109     0.

In [9]:
for word, index in tokenizer.word_index.items():
    vector_value = embedding_dict.get(word)
    if vector_value is not None:
        embedding_matrix[index] = vector_value
embedding_matrix[2]


NameError: ignored

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

output_dim = 100

model = Sequential()
e = Embedding(vocab_size, output_dim, weights=[embedding_matrix], input_length=max_len, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.fit(X_train, y_train, epochs=100, verbose=2)

🔹 **1-(3)** fine tuning glove
* 미세조정 : 사전 학습한 모든 가중치와 더불어 하위 문제를 위한 최소한의 가중치를 추가해 모델을 추가로 학습하는 방법이다. 

* fine tuning 이 필요한 경우 
  * pretrained model 에 데이터셋에 있는 단어가 포함되지 않은 경우 
  * 데이터 집합이 너무 작아서 전체 모델을 훈련시키기 어려운 경우 

* [Mittens 라이브러리로 fine tuning](https://towardsdatascience.com/fine-tune-glove-embeddings-using-mittens-89b5f3fe4c39) 👉 필수
  *  GloVe 임베딩을 fine-tuning 하기 위한 파이썬 라이브러리
  * [github](https://github.com/roamanalytics/mittens)

* [한국어 소설 텍스트 데이터 미세조정 모델 학습 - GPT2](https://m.blog.naver.com/PostView.nhn?isHttpsRedirect=true&blogId=horajjan&logNo=222104684132&categoryNo=120&proxyReferer=) 👉 선택 (glove 모델 예제는 아닙니다. fine-tuning 에 초점을 두어서 참고해주시면 좋을 것 같습니다.)

In [27]:
import csv
import numpy as np

def glove2dict(glove_filename):
    with open(glove_filename) as f:
        reader = csv.reader(f, delimiter=' ', quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed

In [28]:
def glove2dict(glove_filename):
    with open(glove_filename, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter=' ',quoting=csv.QUOTE_NONE)
        embed = {line[0]: np.array(list(map(float, line[1:])))
                for line in reader}
    return embed
glove_path = "glove.6B.50d.txt"
pre_glove = glove2dict(glove_path)

sw = list(stop_words.ENGLISH_STOP_WORDS)
brown_data = brown.words()[:200000]
brown_nonstop = [token.lower() for token in brown_data if (token.lower() not in sw)]
oov = [token for token in brown_nonstop if token not in pre_glove.keys()]

def get_rareoov(xdict, val):
    return [k for (k,v) in Counter(xdict).items() if v<=val]
oov_rare = get_rareoov(oov, 1)
corp_vocab = list(set(oov) - set(oov_rare))

brown_tokens = [token for token in brown_nonstop if token not in oov_rare]
brown_doc = [' '.join(brown_tokens)]
corp_vocab = list(set(oov))

cv = CountVectorizer(ngram_range=(1,1), vocabulary=corp_vocab)
X = cv.fit_transform(brown_doc)
Xc = (X.T * X)
Xc.setdiag(0)
coocc_ar = Xc.toarray()

NameError: ignored

In [29]:
mittens_model = Mittens(n=50, max_iter=1000)
new_embeddings = mittens_model.fit(
    coocc_ar,
    vocab=corp_vocab,
    initial_embedding_dict= pre_glove)

NameError: ignored

In [26]:
newglove = dict(zip(corp_vocab, new_embeddings))
f = open("repo_glove.pkl","wb")
pickle.dump(newglove, f)
f.close()

NameError: ignored

* (참고) word2vec pretrained example

➕ [word2vec 사전학습 모델 -한국어1](http://doc.mindscale.kr/km/unstructured/11.html)

➕ [word2vec 사전학습 - 한국어2](https://monetd.github.io/python/nlp/Word-Embedding-Word2Vec-%EC%8B%A4%EC%8A%B5/#%ED%95%9C%EA%B5%AD%EC%96%B4-word2vec-%EB%A7%8C%EB%93%A4%EA%B8%B0)

### **2️⃣ NER**

👀 **내용 복습** 
* 개체명 인식을 사용하면 코퍼스로부터 어떤 단어가 사람, 장소, 조직 등을 의미하는 단어인지를 찾을 수 있다. 




🔹 **2-(1)** NER task by nltk library


* nltk 에서는 개체명 인식기 (NER chunker) 를 지원하고 있다. 
* ne_chunk 는 개체명을 태깅하기 위해서 앞서 품사 태깅 pos_tag 가 수행되어야 한다. 


📌 [basic code](https://wikidocs.net/30682) 👉 필수 

📌 [BIO 표현, LSTM을 활용한 NER 실습](https://wikidocs.net/24682) 👉 선택




In [30]:
from nltk import word_tokenize, pos_tag, ne_chunk

sentence = "James is working at Disney in London"

tokenized_sentence = pos_tag(word_tokenize(sentence))
print(tokenized_sentence)

[('James', 'NNP'), ('is', 'VBZ'), ('working', 'VBG'), ('at', 'IN'), ('Disney', 'NNP'), ('in', 'IN'), ('London', 'NNP')]


In [31]:
ner_sentence = ne_chunk(tokenized_sentence)
print(ner_sentence)

(S
  (PERSON James/NNP)
  is/VBZ
  working/VBG
  at/IN
  (ORGANIZATION Disney/NNP)
  in/IN
  (GPE London/NNP))


🔹 **2-(2)** NER task by spacy library


* spaCy 는 자연어처리를 위한 파이썬 기반의 오픈 소스 라이브러리로 다음과 같은 기능을 제공한다. 
  * Tokenization 
  * POS tagging 
  * Lemmatization 
  * Sentence Boundary Detection (SBD)
  * Named Entity Recognition (NER)
  * Similarity
  * Text Classification
  * Rule-based Matching
  * Training
  * Serialization

* spaCy 와 NER
  * .ents → .label_


📌 [basic code](https://frhyme.github.io/python-lib/nlp_spacy_1/) 👉 필수 (NER 부분만)

📌 [kaggle_Custom NER using SpaCy](https://www.kaggle.com/code/amarsharma768/custom-ner-using-spacy/notebook) 👉 선택

  * 훈련되지 않은 데이터 세트에 명명된 엔티티를 학습하는 방법 : 이력서 pdf 데이터 활용 
  * manually labelled 

📌 [한국어 NER](https://github.com/monologg/KoBERT-NER) 👉 참고하면 좋을 자료

➕ [참고](http://aispiration.com/nlp2/nlp-ner-python.html)

In [34]:
python -m spacy download en_core_web_md
python -m spacy download en_core_web_lg

SyntaxError: ignored

In [32]:
import spacy

nlp = spacy.load('en_core_web_md')  
tokens = nlp(u'dog cat banana')

for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

OSError: ignored

In [33]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances

nlp = spacy.load('en_core_web_md')  
tokens = nlp(u'dog cat banana')

for token1 in tokens:
    for token2 in tokens:
        sim = cosine_similarity(token1.vector.reshape(1, 300), 
                          token2.vector.reshape(1, 300)
                         )
        print(token1.text, token2.text, sim)

OSError: ignored

###**3️⃣ Dependency Parsing**

👀 **내용 복습** 
* 문장의 전체적인 구성/구조 보다는 각 개별단어 간의 '의존관계' 또는 '수식관계' 와 같은 단어간 관계를 파악하는 것이 목적인 NLP Task
* 문장 해석의 모호성을 없애기 위해 Parsing 을 한다.




🔹 **3-(1)** Dependency Parsing by spacy library


* [basic](https://frhyme.github.io/python-lib/nlp_spacy_1/#navigating-parse-tree) 👉 dependecy parsing 부분만 필수
* .dep_ 메서드



In [36]:
pip install -U pip setuptools wheel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.0 MB/s 
     |████████████████████████████████| 1.2 MB 44.9 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
numba 0.56.2 requires setuptools<60, but you have setuptools 65.4.1 which is incompatible.


In [11]:
one_sentence = tweets['text'][300]
doc = nlp(one_sentence)
spacy.displacy.render(doc, style='ent', jupyter=True)

NameError: ignored

In [12]:
trump_topics = []
for ent in doc.ents:
  if ent.label_ not in ["PERCENT", "CARDINAL", "DATE"]:
      trump_topics.append(ent.text.strip())

NameError: ignored

🔹 **3-(2)** Spacy (kaggle) 

* 캐글 노트북 환경에서 실습해보는 것을 권장드립니다!

* [kaggle_spaCy](https://www.kaggle.com/code/nirant/hitchhiker-s-guide-to-nlp-in-spacy) 👉 필수
  * 도날드 트럼프 트위터 트윗 내용 데이터 분석


👀 **노트북 키포인트** 
  1. spacy.display 메서드를 사용한 NER 시각화 
  2. Tagging 을 통한 트럼프 트윗 분석 : noun_chunks 는 dependency graph를 고려하여, noun phrase를 뽑아준다. 
  3. [spacy Match](https://yujuwon.tistory.com/entry/spaCy-%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0-Rule-based-Matching) : 직접 문장/단어 패턴을 등록하여 parsing
  4. Question and answering task using Dependency Parsing
    * spacy display :  ``style = 'dep'``
    * .dep_


In [3]:
!python -m spacy download en_core_web_lg
nlp = spacy.load('en_core_web_lg')

2022-10-10 11:44:10.005812: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


NameError: ignored

In [7]:
import pandas as pd

In [8]:
tweets = pd.read_csv("../input/all-djtrum-tweets/all_djt_tweets.csv")

FileNotFoundError: ignored

In [6]:
def explain_text_entities(text):
    doc = nlp(text)
    for ent in doc.ents:
        print(f'Entity: {ent}, Label: {ent.label_}, {spacy.explain(ent.label_)}')

explain_text_entities(tweets['text'][9])

one_sentence = tweets['text'][0]
doc = nlp(one_sentence)
spacy.displacy.render(doc, style='ent',jupyter=True)

NameError: ignored